# Data 200: Database Systems and Data Management for Data Analytics


# Take-Home Final Exam

**Dickinson College**<br/>
**Fall 2022**<br/>
Prof. Eren Bilen<br/>
<font color='red'>**Due Date:** Friday December 16, 12pm (noon) </font>
---
Enter your name in the markdown cell below.

# Name: Chloe Ho

In [366]:
from selenium import webdriver
import pandas as pd
import re
import time 
import random
import matplotlib.pyplot as plt

# Scraping task

On this exam, you will scrape reviews from BestBuy.com. The item for which you will scrape reviews for will depend on the first letter of your last name. Check the table below (both letters inclusive) for your assigned item:

|First letter of your last name | Item/link to reviews on BestBuy.com
| :---        |    :---   |
|A-G | https://www.bestbuy.com/site/reviews/microsoft-xbox-series-x-1tb-console-black/6428324?variant=A&skuId=6428324
|H-M | https://www.bestbuy.com/site/reviews/apple-airpods-with-charging-case-2nd-generation-white/6084400?variant=A&skuId=6084400
|N-S | https://www.bestbuy.com/site/reviews/samsung-65-class-7-series-led-4k-uhd-smart-tizen-tv/6401722?variant=A&skuId=6401722
|T-Z | https://www.bestbuy.com/site/reviews/nintendo-switch-32gb-lite-turquoise/6257139?variant=A&skuId=6257139


Your goal is to scrape 2,000 reviews that were posted in the reviews page of your assigned item. The format of your output should be a dataframe with the following columns/format:
<br>
![title](ss1.png)

where
```
username: the user name of the reviewer
review: the review content
rating: the rating (out of 5) left by the reviewer
helpful: the number of votes received on the review indicating that the review is helpful
unhelpful: the number of votes received on the review indicating that the review is unhelpful
net: |helpful - unhelpful| i.e., the absolute value of the difference between "helpful" and "unhelpful" votes
nethelpful: helpful - unhelpful i.e., the difference between "helpful" and "unhelpful" votes
total: total number of votes (helpful + unhelpful) received
length: length of the review, defined as the number of characters in a review (including whitespace)
```

Include the `.ipynb` and `.md` versions of your work + the following two scatterplots as part of your output. Comment on what you observe from the scatterplots. Are you surprised?
* scatterplot with `length` on the x axis, `total` on the y axis
* scatterplot with `length` on the x axis, `nethelpful` on the y axis

Commit and push on Github once you finish.

Hints: 
* You will need to scrape multiple pages. Make sure to wait at least 2 seconds between each page, e.g., using `time.sleep(random.uniform(2, 3))`.
* Trim away `\n` from review strings as it contains no useful information, and can cause bugs in regex.
* Make sure to get the position of the "next Page" button, as its position can change dynamically depending on the page number.

Good luck!

In [367]:
driver = webdriver.Chrome('/Users/hochl/Downloads/chromedriver')

/var/folders/2f/00sdrmc55c31t4b6_w_60sl80000gn/T/ipykernel_5327/1548921077.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/hochl/Downloads/chromedriver')


In [368]:
driver.get('https://www.bestbuy.com/site/reviews/apple-airpods-with-charging-case-2nd-generation-white/6084400?variant=A&skuId=6084400')

In [381]:
def get_reviewid():
    # get comment ids that appear on the page
    ids = driver.find_elements('xpath',"//*[contains(@id,'ugc-line-clamp-reviews-')]")
    review_ids = []
    for i in ids:
        review_ids.append(i.get_attribute('id'))
    return review_ids

def get_reviews():

    reviews = []
    n=1

    for x in review_ids:
        # Extract username from each user
        username_element = driver.find_element("xpath", "//*[@id='reviews-accordion']/div[1]/ul/li["+str(n)+"]/div/div[2]/div[2]/div[1]/span/span/button/div/strong")
        username = username_element.text

        # Extract review
        review_element = driver.find_element("xpath", '//*[@id="'+x+'"]')
        review = review_element.text.strip("\n")
        
        # Extract rating 
        rating_element = driver.find_element("xpath", "//*[@id='reviews-accordion']/div[1]/ul/li["+str(n)+"]/div/div[2]/div[1]/div")
        rating = rating_element.text
        rating = re.findall("\d", rating)[0]

        # Extract helpful votes
        helpful_element = driver.find_element("xpath", "//*[@id='reviews-accordion']/div[1]/ul/li["+str(n)+"]/div/div[2]/div[6]/div/div/div[1]/button[1]")
        helpful = helpful_element.text
        helpful = re.findall("\d+", helpful)[0]

        # Extract unhelpful votes
        unhelpful_element = driver.find_element("xpath", "//*[@id='reviews-accordion']/div[1]/ul/li["+str(n)+"]/div/div[2]/div[6]/div/div/div[1]/button[2]")
        unhelpful = unhelpful_element.text
        unhelpful = re.findall("\d+", unhelpful)[0]

        # Take absolute value of the difference between "helpful" and "unhelpful" votes
        net = abs(int(helpful) - int(unhelpful))

        # Take the difference between "helpful" and "unhelpful" votes
        nethelpful = int(helpful) - int(unhelpful)

        # Total number of votes (helpful + unhelpful) received
        total = int(helpful) + int(unhelpful)
        
        # The length of the review
        length = len(review)
        
        reviews.append([username,review,rating,helpful,unhelpful,net,nethelpful,total,length])
        
        n+=1
    
    return reviews

def get_loc():
    num = len(driver.find_element('xpath','//*[@id="reviews-accordion"]').text.split('\n'))
    attributes = []
    location = []
    for m in range(20):
        try:
            attributes.append(driver.find_element('xpath',"//*[@id='reviews-accordion']/div[2]/div/div[2]/ul/li["+str(m)+"]/a") \
                              .get_attribute('title'))
            if driver.find_element('xpath',"//*[@id='reviews-accordion']/div[2]/div/div[2]/ul/li["+str(m)+"]/a") \
                              .get_attribute('title') == 'next Page':
                location.append(m)
            return location[0]
        except:
            attributes.append('.')

In [382]:
num = len(driver.find_element('xpath','//*[@id="reviews-accordion"]').text.split('\n'))
num

273

In [ ]:
df_out = pd.DataFrame()

for j in range(105):
    
    try:

        # get comment ids
        review_ids = get_reviewid()

        # get comments
        reviews = get_reviews()

        # df with the current page data
        df = pd.DataFrame(reviews,columns=['username','review','rating','helpful','unhelpful','net','nethelpful','total','length'])

        # df_out with the entire data
        df_out = pd.concat([df_out,df])

        # wait between 2-3 second
        time.sleep(random.uniform(2,3))

        # find the location of the next page button, then click on it!
        button = driver.find_element('xpath','//*[@title="next Page"]')
        button.click()
    
    except: 
        pass

    

In [ ]:
df_out.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(7,4)) # change figure size

plt.scatter(df_out.length, df_out.total, alpha=0.5, color = 'blue')
plt.title("Scatter plot of Review Length and Total Number of Votes")

plt.xlabel('Review length')
plt.ylabel('Total votes')

plt.show()

In [ ]:
plt.figure(figsize=(7,4)) # change figure size

plt.scatter(df_out.length, df_out.nethelpful, alpha=0.5)
plt.title("Scatter plot of Review Length and Review Net Helpful")

plt.xlabel('Review length')
plt.ylabel('Net helpful')

plt.show()

The scatter plot shows that there is no linear relationship between the length of the review and the total of votes  received by the review/nethelpful indicator. There are quite a few outliers: a review has more than 4000 characters but received 0 votes and 0 nethelpful while a-less-than-1000-character review receives around 550 total votes & more than 200 nethelpful scores. Most of the reviews receive 0 to 50 total votes and the average nethelpful score is 0.